In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

dir='/content/drive/MyDrive/Datasets2023/WHOI-Plankton/' #specify your dir here

IMAGES = np.load(dir+'IMAGES.npy')
IM_class = np.load(dir+'IM_class.npy')
class_labels = np.load(dir+'class_labels.npy')

N_classes=len(np.unique(class_labels))

In [3]:
keep_classes=np.array(['Leptocylindrus', 'Chaetoceros', 'dino30' , 'Rhizosolenia', 'Guinardia_delicatula',
                       'Cerataulina', 'Cylindrotheca', 'Skeletonema', 'Thalassiosira', 'Ciliate_mix'])
# reshape the images for input to CNN
X= np.expand_dims(IMAGES, axis=3)
# create the one-hot encoding of the training labels
Y = keras.utils.to_categorical(class_labels, len(keep_classes))

In [4]:
pip install codecarbon

In [5]:
dir='/content/drive/MyDrive/Datasets2023/WHOI-Plankton/' #specify your dir here

IMAGES = np.load(dir+'IMAGES.npy')
IM_class = np.load(dir+'IM_class.npy')
class_labels = np.load(dir+'class_labels.npy')

N_classes=len(np.unique(class_labels))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import AdamW
from codecarbon import EmissionsTracker
import time


input_shape = (64, 64, 1)
output_shape = len(keep_classes)

val_split = 0.2
ind_val = (np.random.rand(X.shape[0]) < val_split)

X_val, Y_val = X[ind_val], Y[ind_val]
X_train, Y_train = X[~ind_val], Y[~ind_val]

def create_rnn_model(input_shape, output_shape):
    x = keras.layers.Input(shape=input_shape)

    y = keras.layers.Conv2D(8, (3,3), activation='relu', padding='same')(x)
    y = keras.layers.Conv2D(8, (3,3), activation='relu', padding='same')(y)
    y = y + x
    y = keras.layers.MaxPooling2D(pool_size=(2,2))(y)

    y = keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(y)
    y0 = y
    y = keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(y)
    y = keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(y)
    y = y0 + y
    y = keras.layers.MaxPooling2D(pool_size=(2,2))(y)

    y = keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(y)
    y1 = y
    y = keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(y)
    y = keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(y)
    y = y1 + y
    y = keras.layers.MaxPooling2D(pool_size=(2,2))(y)

    y = keras.layers.Conv2D(64, (3,3), activation='relu', padding='same')(y)
    y2 = y
    y = keras.layers.Conv2D(64, (3,3), activation='relu', padding='same')(y)
    y = keras.layers.Conv2D(64, (3,3), activation='relu', padding='same')(y)
    y = y2 + y
    y = keras.layers.MaxPooling2D(pool_size=(2,2))(y)

    y = keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')(y)
    y3 = y
    y = keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')(y)
    y = keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')(y)
    y = y3 + y
    y = keras.layers.MaxPooling2D(pool_size=(2,2))(y)

    y = keras.layers.Conv2D(256, (3,3), activation='relu', padding='same')(y)

    y = keras.layers.GlobalAveragePooling2D()(y)
    y = keras.layers.Dropout(0.5)(y)
    y = keras.layers.Dense(output_shape, activation='softmax')(y)

    model = keras.models.Model(inputs=x, outputs=y)


    model.compile(optimizer=AdamW(learning_rate=0.001, weight_decay=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


input_shape = (64,64,1)
output_shape = 10
batch_size = 512
# for 5 runs
n_epochs = 25
num_runs = 5


for i in range(num_runs):
    tracker = EmissionsTracker(allow_multiple_runs=True)
    tracker.start()


    model = create_rnn_model(input_shape, output_shape)
    hist = model.fit(X_train, Y_train, epochs=n_epochs, batch_size=batch_size, validation_data=(X_val, Y_val), verbose=1)


    emissions = tracker.stop()



[codecarbon WARNING @ 03:58:54] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 03:58:54] [setup] RAM Tracking...
[codecarbon INFO @ 03:58:54] [setup] CPU Tracking...
[codecarbon WARNING @ 03:58:54] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 03:58:55] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 03:58:55] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 03:58:55] [setup] GPU Tracking...
[codecarbon INFO @ 03:58:55] No GPU found.
[codecarbon INFO @ 03:58:55] >>> Tracker's metadata:
[codecarbon INFO @ 03:58:55]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 03:58:55]   Python version: 3.11.11
[codecarbon INFO @ 03:58:55]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 03:5

Epoch 1/25


[codecarbon INFO @ 03:59:11] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:59:11] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:59:11] 0.000197 kWh of electricity used since the beginning.


  3/101 ━━━━━━━━━━━━━━━━━━━━ 8:38 5s/step - accuracy: 0.1172 - loss: 2.3028

[codecarbon INFO @ 03:59:26] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:59:26] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:59:26] 0.000394 kWh of electricity used since the beginning.


  6/101 ━━━━━━━━━━━━━━━━━━━━ 8:03 5s/step - accuracy: 0.1195 - loss: 2.2986

[codecarbon INFO @ 03:59:41] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 03:59:41] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 03:59:41] 0.000591 kWh of electricity used since the beginning.


  7/101 ━━━━━━━━━━━━━━━━━━━━ 7:46 5s/step - accuracy: 0.1209 - loss: 2.2961